# Linear Regression - ECommerce

Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

In [1]:
# Lets start a Spark Session

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [2]:
# Import the Linear Regression library

from pyspark.ml.regression import LinearRegression

## Read the dataset

In [3]:
# We will now read the dataset

data = spark.read.csv("resources/Ecommerce_Customers.csv", inferSchema=True, header=True)

# Print the schema of the data frame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [4]:
# View the initial rows of the data
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [5]:
# Check the first row
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [6]:
# Check the individual column values from the first row
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up the dataframe for Machine Learning

A few things we need to do before Spark can accept the data! It needs to be in the form of two columns 
("label","features")

In [7]:
# Import VectorAssembler and Vectors

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
# Check the columns
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
# We will use the numerical columns only for now

assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 
                                       'Length of Membership', 'Yearly Amount Spent'], 
                            outputCol="features")

# Now we will transform the data

output = assembler.transform(data)

In [10]:
# If we do a printSchema we will see a feautures column got added which is actually a dense vector

output.printSchema()

# See the first row
output.head()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))

In [11]:
# Final data for regression

final_data = output.select("features", 'Yearly Amount Spent')

In [12]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

## Perform the Regression

In [13]:
# Perform the train, test split

train_data, test_data = final_data.randomSplit([0.7, 0.3])

train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                358|
|   mean|   503.753052245877|
| stddev|  79.99876666138665|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [14]:
# Create the Linear Regression Model Object

lr = LinearRegression(labelCol='Yearly Amount Spent')

# Fit the model to the data and call this model
lrModel = lr.fit(train_data,)

In [15]:
# Get the coefficients and intercept

print(f"coefficients : {lrModel.coefficients}")
print("\n")
print(f"Intercept : {lrModel.intercept}")

coefficients : [2.611644411365809e-11,3.631740456672849e-11,2.71187365504184e-12,5.724795999535572e-11,0.9999999999990773]


Intercept : -1.1431701332618505e-09


In [20]:
# Now we will evaluate this model 

test_results = lrModel.evaluate(test_data)

In [21]:
# Get the residuals, RMSE and Adjusted R2

test_results.residuals.show()

print(f"RMSE : {test_results.rootMeanSquaredError}")
print(f"Adjusted R2 : {test_results.r2adj}")

+--------------------+
|           residuals|
+--------------------+
|5.115907697472721...|
|6.30961949354969E-12|
|1.716671249596402...|
|1.307398633798584...|
|-6.02540239924564...|
|1.443822839064523...|
|-1.02318153949454...|
|3.012701199622824...|
|-1.21644916362129...|
|4.206412995699793...|
|3.240074875066057...|
|-1.08002495835535...|
|5.627498467219993...|
|4.604316927725449...|
|1.028865881380625...|
|-6.42330633127130...|
|1.705302565824240...|
|                 0.0|
|-6.48014975013211...|
|-5.45696821063756...|
+--------------------+
only showing top 20 rows

RMSE : 9.708843191632096e-12
Adjusted R2 : 1.0


In [22]:
# Now if we have to run these as predictions on an unseen data

unlabeled_data = test_data.select('features')

predictions = lrModel.transform(unlabeled_data)

predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...| 282.4712457199094|
|[30.5743636841713...|442.06441375805935|
|[30.7377203726281...| 461.7807421962127|
|[30.8794843441274...|490.20659998485337|
|[31.0662181616375...| 448.9332932076804|
|[31.1695067987115...|427.35653080227837|
|[31.4252268808548...| 530.7667186547629|
|[31.5316044825729...|436.51560572935955|
|[31.5702008293202...|  545.945492141417|
|[31.6610498227460...|416.35835357989663|
|[31.7216523605090...| 347.7769266318694|
|[31.7242025238451...| 503.3878872879616|
|[31.8124825597242...| 392.8103449837916|
|[31.8186165667690...|446.41867337013105|
|[31.8209982016720...|424.67528101320306|
|[31.8279790554652...|440.00274754694794|
|[31.8530748017465...| 459.2851234623518|
|[31.8627411090001...| 556.2981411740467|
|[31.8648325480987...|439.89128047682016|
|[31.8854062999117...|390.10327297248097|
+--------------------+------------